# **AVISO EN CASO DE ERRORES**

En caso de que la librería de pandas-profiling de errores de ejecución (lo cual es bastante común), ejecute estas líneas de código y reinicie el entorno de ejecución para que funcione la parte del profiling correctamente en caso de que se quieran ejecutar dichos apartados.

In [ ]:
%%capture
!pip uninstall -y pandas-profiling
!pip install pandas-profiling

# Descripción del Colab

En este Colab se muestran los procesos de limpieza de datos y profiling de los distintos datasets que hemos empleado para el laboratorio de Minería de Datos. 

La explicación de los procesos realizados para cada dataset se muestran separados a continuacion en este Colab.

### Librerías empleadas en este fichero

In [ ]:
%%capture
!pip install deep_translator

In [ ]:
import re
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport
from pandas.io.formats.style_render import DataFrame
from google.colab import files
import io
from deep_translator import GoogleTranslator
import sys

# Carga de datos
Ejecución de código para seleccionar los archivos que les hemos otorgado paa subirlos en el Colab.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Dataset 1 - Ventas y precios nacionales
Este dataset contiene la información de las ventas y el consumo nacional de diferentes frutas y hortalizas desde 2018 hasta noviembre de 2020. Se va a utilizar el dataset llamado '*Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt*'

Este dataset se utiliza mezclado con otro que veremos más adelante para las hipótesis 3 y 4: 

*-Durante una pandemia se produce un incremento del precio de los productos agrícolas.*

*-Durante una pandemia se produce un decremento del consumo de los productos agrícolas.*

### Descripción de valores y columnas en el dataset final
Una vez limpio el dataset tendremos un total de 4 columnas:

*   **fecha**: es el mes y año en el que se estan mostrando los datos de la fila. Esta en formato "Mes/Año".
*   **producto**: es el nombre de la fruta u hortaliza.
*   **precio_medio_kg**: es el precio medio de la fruta u hortaliza ese mes.
*   **consumo_per_capita**: es la relación entre el total de los habitantes del país y la cantidad de producto que han consumido ese mes.

Lo primero que vamos a hacer es cargar los datos en el dataframe llamado "dataframe_1".

In [ ]:
dataframe_1 = pd.read_csv("Dataset1.- DatosConsumoAlimentarioMAPAporCCAA.txt", sep="|")

## Limpieza de columnas

Aquí vamos a eliminar las columnas que no nos aportan nada para nuestro objetivo, que en este caso sería ver el precio de unos productos y el consumo de estos en España. En este caso sólo vamos a querer las columnas "Año", "Mes", "CCAA" (esta la eliminaremos después porque la necesitamos para algo antes), "Producto", "Precio medio kg" y "Consumo per capita".

In [ ]:
del dataframe_1["Penetración (%)"]
del dataframe_1["Valor (miles de €)"]
del dataframe_1["Gasto per capita"]
del dataframe_1["Volumen (miles de kg)"]
del dataframe_1["Unnamed: 10"]
del dataframe_1["Unnamed: 11"]

Aquí cambiamos el nombre de las columnas "Precio medio kg" y "Consumo per capita" para reemplazarle los espacios del nombre por "_". Además pondremos en minúscula el nombre de estas dos columnas anteriores y la columna de "producto". Esto es para manipular los datos de forma más comoda en adelante.

In [ ]:
dataframe_1.rename({'Precio medio kg': 'precio_medio_kg'}, axis=1, inplace=True)
dataframe_1.rename({'Consumo per capita': 'consumo_per_capita'}, axis=1, inplace=True)
dataframe_1.rename({'Producto': 'producto'}, axis=1, inplace=True)

dataframe_1.head()

,Año,Mes,CCAA,producto,precio_medio_kg,consumo_per_capita
0,2018,Enero,Total Nacional,TOTAL PATATAS,"0,78","2,38"
1,2018,Enero,Total Nacional,PATATAS FRESCAS,"0,69","1,74"
2,2018,Enero,Total Nacional,PATATAS CONGELADAS,"1,21","0,09"
3,2018,Enero,Total Nacional,PATATAS PROCESADAS,"5,02","0,11"
4,2018,Enero,Total Nacional,T.HORTALIZAS FRESCAS,"1,79","4,6"


A continuación se ejecuta la siguiente línea de código para saber si hay elementos nulos (vemos que no los hay).

In [ ]:
dataframe_1.isnull().values.all()

False

## Limpieza de filas
Primero vamos a quitar los datos de las comunidades autónomas dejando solo los datos nacionales. Esto es debido a que los datos de comunidades autónomas los tenemos en otro dataset ya más completo.
A continuación eliminamos la columna de CCAA, como dijimos previamente, porque ya no la necesitamos.

In [ ]:
df1_final = dataframe_1.drop(dataframe_1[dataframe_1.CCAA != "Total Nacional"].index)
del df1_final["CCAA"]

A continuación primero se cambia el nombre del producto "TOTAL PATATAS" a "PATATAS" debido a que hay muchos tipos de patatas y este los aúna todos. Entonces una vez cambiado el nombre eliminamos los demas productos referentes a patatas específicas.
Finalmente eliminamos los productos que aúnan multiples productos en uno solo estilo "Total hortalizas frescas" que lo único que hace es sumar los datos de todas las hortalizas.

In [ ]:
df1_final['producto'].replace({'TOTAL PATATAS': 'PATATAS'}, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "PATATAS FRESCAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "PATATAS CONGELADAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "PATATAS PROCESADAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "T.HORTALIZAS FRESCAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "OTR.HORTALIZAS/VERD."].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "VERD./HORT. IV GAMA"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "T.FRUTAS FRESCAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "OTRAS FRUTAS FRESCAS"].index, inplace=True)
df1_final.drop(df1_final[df1_final.producto == "FRUTAS IV GAMA"].index, inplace=True)

Tras eliminar todas estas filas problemáticas, se decide reindexar el dataset

In [ ]:
df1_final = df1_final.reset_index(drop = True)

Ahora vamos a unir las columnas de Año y Mes en una columna llamada "fecha" con el formato "Mes(en número)/Año". Para ello lo primero que vamos a hacer es cambiar los meses, que están escritos con letra, a número.

In [ ]:
for index, value in df1_final['Mes'].iteritems():
    mapping = {
        'Enero': 1,
        'Febrero': 2,
        'Marzo': 3,
        'Abril': 4,
        'Mayo': 5,
        'Junio': 6,
        'Julio': 7,
        'Agosto': 8,
        'Septiembre': 9,
        'Octubre': 10,
        'Noviembre': 11,
        'Diciembre': 12
        }
    df1_final.loc[index, 'Mes'] = mapping[value]

In [ ]:
df1_final.head()

,Año,Mes,producto,precio_medio_kg,consumo_per_capita
0,2018,1,PATATAS,"0,78","2,38"
1,2018,1,TOMATES,"1,69","0,88"
2,2018,1,CEBOLLAS,"1,01","0,57"
3,2018,1,AJOS,"4,17","0,06"
4,2018,1,COLES,"1,08","0,15"


A continuación vamos a juntar las dos columnas (Año y Mes) en una sóla columna (fecha) con el formato "Mes/Año" y finalmente se reposiciona la columna "fecha" a la primera posición.

In [ ]:
df1_final['fecha'] = df1_final['Mes'].astype(str) + '/' + df1_final['Año'].astype(str)

columns = df1_final.columns.tolist()
columns.insert(0, columns.pop(columns.index('fecha')))
df1_final = df1_final.reindex(columns=columns)

del df1_final["Año"]
del df1_final["Mes"]

In [ ]:
df1_final.head()

,fecha,producto,precio_medio_kg,consumo_per_capita
0,1/2018,PATATAS,"0,78","2,38"
1,1/2018,TOMATES,"1,69","0,88"
2,1/2018,CEBOLLAS,"1,01","0,57"
3,1/2018,AJOS,"4,17","0,06"
4,1/2018,COLES,"1,08","0,15"


Aquí se comprueba qué tipo es cada columna por si hay que cambiar alguna.

In [ ]:
df1_final.dtypes

fecha                 object
producto              object
precio_medio_kg       object
consumo_per_capita    object
dtype: object

Finalmente se decide que si y se cambia el tipo de la columna "Mes" a string, el de la columna "Producto" a string, el de la columna "Precio_medio_kg" a float y el de la columna "Consumo_per_capita" a float.

In [ ]:
df1_final['fecha'] = df1_final['fecha'].astype('string')
df1_final['producto'] = df1_final['producto'].astype('string')

df1_final['precio_medio_kg'] = df1_final['precio_medio_kg'].str.replace(',', '.').astype(float)
df1_final['consumo_per_capita'] = df1_final['consumo_per_capita'].str.replace(',', '.').astype(float)

df1_final.dtypes

fecha                  string
producto               string
precio_medio_kg       float64
consumo_per_capita    float64
dtype: object

## Descarga de dataset resultante
Se tranforma el dataframe a csv separando las celdas por el símbolo ";", que es el símbolo que lee excel por defecto como separador.

In [ ]:
df1_final.to_csv('df_Spain_data.csv', sep = ';', index = False)

files.download('df_Spain_data.csv')

## Profiling del dataset 1

Hacemos profiling del primer dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(df1_final, title="Consumo alimentario en España", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Dataset 2 - Consumo comunidades

Este dataset está centrado en los precios, volumen y valores de ventas mensuales de ciertos productos agrícolas en las comunidades de Andalucía, Madrid, Castilla-La Mancha, Castilla y León, y Cataluña.

La principal intención de este dataset es ser empleado para la hipótesis 2: 

*Las zonas con mayor densidad de población tienen un mayor número de ventas de productos agrícolas.*

### Descripción de valores y columnas en el dataset final
Dentro de este dataset nos encontramos un total de 22 columnas:

*   **Producto**: lista de los productos contenidos en el dataset.
*   **Fecha**: muestra el mes y el año de los datos.

En las siguientes columnas nos hemos ahorrado en explicarlas por separado, pues son varias columnas que muestran la misma información pero modificando de qué comunidad (COM) provienen los datos. Por lo que se pueden resumir en lo siguiente:
*   **Precio COM (€/Kg)**: Estas columnas muestran el valor del precio medio de un producto en una fecha para cada región.
*   **Valor COM (miles €)**: Estas columnas muestran el valor total de las ganancias de las ventas de un producto en una fecha para cada región.
*   **Volumen COM (miles kg)**: Estas columnas muestran el volumen total de las ventas de un producto en una fecha para cada región.
*   **Población COM**: Estas columnas muestran la población total de una región con valores anuales.

### Proceso de limpieza de los datos

En este apartado explicaremos qué procesos se han seguido para limpiar los datos, de dónde provienen estos y qué información contenían originalmente.

Los datos originales de los datos con los productos agrícolas provienen en 3 archivos excel, cuya información viene dividida por meses en hojas de excel. Junto a esto, tenemos otro dataset que contiene la información de la población española por comunidades.

Para poder leer correctamente los datos de los primeros 3 archivos se crea un bucle *for* que itere entre los archivos con otro bucle anidado que itere por las hojas dentro de estos.

Para poder seleccionar los datos que nos interesan guardamos en la primera iteración del bucle la lista de los productos que nos interesen en la lista llamada ***products***. Esta lista nos sirve para poder hacer un *merge* entre la lista *products* y la hoja con todos los datos que vamos leyendo. De esta manera almacenamos en la variable ***mes_aux*** los productos con su datos de ventas para el mes que estemos leyendo.

En cuanto a los datos de la población, debemos leer el archivo almacenado en **poblacion** iterativamente de forma inversa, tal que para cada comunidad autónoma tenemos que asignar la población de esta desde 2018 hasta 2020. Se recorre al revés, con el índice i, debido a que la información en el fichero original viene desde el año más reciente hasta el último. De este fichero sólo nos fijamos en los datos de cada comunidad, filtrándolos uno a uno.

Tras filtrar los datos del mes en cuestión, eliminamos los duplicados que haya. Esto se debe a que los datos originales muestra información sobre distintos tipos de tomates, pimientos o judías verdes, entre otros, con la única distinción visual de incluírlos en subapartados (como productos frescos, de conserva o congelados) que no podemos diferenciar a través de código. Por esto, se decide eliminar los duplicados, manteniendo únicamente la primera aparición de los datos, ya que sabemos que son los datos de las hortalizas y frutas frescas, no de conserva ni congelados.

In [ ]:
# Declaración de variables
archivos = ['2018datosmensualesdelpaneldeconsumoalimentarioenhogares_tcm30-520451_tcm30-520451.xlsx',
        '2019datosmensualesdelpaneldeconsumoalimentarioenhogares_tcm30-5204501_tcm30-520450.xlsx',
        '2020-datos-mensuales-panel-hogares-ccaa-rev-nov2021_tcm30-540244.xlsx']
poblacion = '2915bsc.csv'

# Indice para avanzar en el fichero de forma inversa
i = 2

lista_mes = []
lista_mes_2020 = []
meses = {'Enero':'01', 'Febrero':'02', 'Marzo':'03', 'Abril':'04', 'Mayo':'05', 'Junio':'06', 'Julio':'07',
         'Agosto':'08', 'Septiembre':'09', 'Octubre':'10', 'Noviembre':'11', 'Diciembre':'12'}
first = True
pob = pd.read_csv(poblacion, sep=';', encoding='latin-1')
products = pd.DataFrame(columns = ['CONSUMO EN HOGARES'])
mes_aux = pd.DataFrame(columns = ['Producto', 'Precio AND (€/Kg)', 'Precio MAD (€/Kg)', 'Precio CLM (€/Kg)', 'Precio CYL (€/Kg)',
             'Precio CAT (€/Kg)', 'Valor AND (miles €)', 'Valor MAD (miles €)', 'Valor CLM (miles €)', 
             'Valor CYL (miles €)', 'Valor CAT (miles €)', 'Volumen AND (miles kg)', 'Volumen MAD (miles kg)',
             'Volumen CLM (miles kg)', 'Volumen CYL (miles kg)', 'Volumen CAT (miles kg)', 'Poblacion AND',
             'Poblacion MAD', 'Poblacion CLM', 'Poblacion CYL', 'Poblacion CAT'])

# Bucle para la limpieza y obtención de los datos de los datasets originales
for file in archivos:
  año = file[0:4]
  condition = True
  
  # Diferenciamos entre 2018/19 y 2020 al cambiar de formato para nombrar las hojas del excel
  if año == '2020':
    meses = {k.lower(): v for k, v in meses.items()}
    condition = False

  for mes in meses:
    xl = pd.read_excel(file, sheet_name = mes)
    mes_2020 = mes_aux = pd.DataFrame(columns = ['Producto', 'Fecha', 'Precio AND (€/Kg)', 
                                                 'Precio MAD (€/Kg)', 'Valor AND (miles €)', 
                                                 'Valor MAD (miles €)', 'Volumen AND (miles kg)', 
                                                 'Volumen MAD (miles kg)'])
    # Cogemos los productos que nos interesan
    if mes == 'Enero' and first:
      products['CONSUMO EN HOGARES'] = xl.iloc[420:465]['CONSUMO EN HOGARES']
      first = False
    
    # Juntamos el dataset base con la lista de productos con un merge para obtener solo los datos de interés
    first_month = pd.merge(products, xl, on='CONSUMO EN HOGARES', how='left')
    mes_aux[['Producto', 'Precio AND (€/Kg)', 'Precio MAD (€/Kg)', 'Precio CLM (€/Kg)', 'Precio CYL (€/Kg)',
             'Precio CAT (€/Kg)', 'Valor AND (miles €)', 'Valor MAD (miles €)', 'Valor CLM (miles €)', 
             'Valor CYL (miles €)', 'Valor CAT (miles €)', 'Volumen AND (miles kg)', 'Volumen MAD (miles kg)',
             'Volumen CLM (miles kg)', 'Volumen CYL (miles kg)', 'Volumen CAT (miles kg)']] = first_month[['CONSUMO EN HOGARES', 
             'Unnamed: 40', 'Unnamed: 46', 'Unnamed: 52', 'Unnamed: 64', 'Unnamed: 10',
             'Unnamed: 41', 'Unnamed: 47', 'Unnamed: 53', 'Unnamed: 65', 'Unnamed: 11',
             'Unnamed: 42', 'Unnamed: 48', 'Unnamed: 54', 'Unnamed: 66', 'Unnamed: 12',]]
    
    mes_aux['Fecha'] = meses[mes]+'/'+año

    # Filtramos los datos de la población de cada comunidad y los almacenamos
    mes_aux['Poblacion AND'] = pob.iloc[3+i]['Total'] 
    mes_aux['Poblacion MAD'] = pob.iloc[39+i]['Total']
    mes_aux['Poblacion CLM'] = pob.iloc[24+i]['Total']
    mes_aux['Poblacion CYL'] = pob.iloc[21+i]['Total']
    mes_aux['Poblacion CAT'] = pob.iloc[27+i]['Total']

    # Limpieza de duplicados
    mes_aux = mes_aux.drop_duplicates(subset=['Producto'], keep='first')

    # Unimos los datos de cada mes y los almacenamos en un dataset aparte
    lista_mes.append(mes_aux)
    dataset_comunidades = pd.concat(lista_mes)
  i-=1

# Limpieza de nulos y normalización de los datos
dataset_comunidades = dataset_comunidades.convert_dtypes()
dataset_comunidades = dataset_comunidades.fillna(0.0)

# Nos quitamos los valores que no nos interesan
not_wanted = ['T.FRUTAS FRESCAS', 'T.HORTALIZAS FRESCAS', 'OTRAS FRUTAS FRESCAS', 
              'OTR.HORTALIZAS/VERD.', 'VERD./HORT. IV GAMA']
ds_clean = dataset_comunidades

for product in not_wanted:
  ds_clean = ds_clean.drop(ds_clean[ds_clean['Producto'] == product].index)

ds_clean = ds_clean.round(decimals=3)
ds_clean.reset_index(drop=True)

## Descarga de dataset resultante
Se tranforma el dataframe a csv separando las celdas por el símbolo ";", que es el símbolo que lee excel por defecto como separador.

In [ ]:
# Ejecute estas líneas de código si quiere descargar el fichero con el dataset resultante

ds_clean.to_csv('df_Spain_comunidades.csv', sep = ';', index = False)

files.download('df_Spain_comunidades.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Profiling del dataset 2

Hacemos profiling del segundo dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
# Profiling of our dataset
profile = ProfileReport(ds_clean, title="Andalucia-Madrid", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Dataset 4 - Comercio Exterior de España

##Descripción del Dataset 4


In [ ]:
df = pd.read_csv("Dataset4.- Comercio Exterior de España.txt", sep ="|")

En este Dataset nos encontramos con importaciones y exportaciones de frutas y hortalizas en España desde 2018 hasta noviembre de 2020. Estos datos vienen detallados por meses, incluyendo un resumen para cado producto resumiendo su importación y exportación en un año desde enero hasta diciembre.
Entrando en detalle, en este dataset nos encontramos con: 
primero con la fecha del dato, a continuación, el país extranjero que 

*   Period: la fecha
*   Reporter: pais extranjero que importa/exporta
*   Partner: España
*   Product: el porducto que importamos/exportamos
*   Flow: si es una operación de importación o exportación
*   Indicators: si medimos el producto por su valor monetario o por su peso
*   Value: valor asociado a la anterior característica

##Limpieza de Datos
En primer lugar nos hemos dado cuenta de la redundancia de filas empleada para indicarnos el valor en euros del producto importado/exportado y su valor en peso. El fichero original utilizaba dos filas para ambos datos y hemos conseguido unir ambas filas para reducir el tamaño del dataset a la mitad. El resultado de este cambio nos ha originado dos nuevas columnas:

*   Value_in_Euros: valor en € del producto importado/exportado
*   Quantity_in_100kg: peso del producto medido en 100 kg

A su vez, hemos eliminado las columnas Indicators y Value.

Anteriormente en Value hemos reemplazado ':' por un 0 para no tener problemas. Porque cuando no había registros de que hubiese importación o exportación, el dataset almacenaba ':'.

Las dos nuevas columnas, que hemos comentado con anterioridad, necesitan sufrir una transformación de string a entero y ahí nos hemos dado cuenta que algunos datos numéricos tienen un espacio entre sus números (por ejemplo 270 934), por lo que hemos necesitado reemplazar ' ' por ''. A continuación hemos hecho su transformación 



In [ ]:
df['FLOW'] = df['FLOW'].astype('category')
df['REPORTER'] = df['REPORTER'].astype('category')
df.Value= df.Value.replace({":":0})

translator = GoogleTranslator(source="en", target="es")

counts1 = df.groupby("INDICATORS").size()
df1 = df[(df.INDICATORS == "VALUE_IN_EUROS")]
df2 = df[(df.INDICATORS == "QUANTITY_IN_100KG")]
df2.index=df1.index
df1.insert(7, "VALUE_IN_EUROS", df1.Value, True)
df1.insert(8, "QUANTITY_IN_100KG", df2.Value, True)

df1.VALUE_IN_EUROS= df1.VALUE_IN_EUROS.replace({' ':''}, regex=True)
df1.QUANTITY_IN_100KG= df1.QUANTITY_IN_100KG.replace({' ':''}, regex=True)
df1['VALUE_IN_EUROS'] = df1['VALUE_IN_EUROS'].astype(int)
df1['QUANTITY_IN_100KG'] = df1['QUANTITY_IN_100KG'].astype(int)


del(df1['INDICATORS'])
del(df1['Value'])

df1.isnull().values.any()

<ipython-input-69-f05fc7974a4d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['VALUE_IN_EUROS'] = df1['VALUE_IN_EUROS'].astype(int)
<ipython-input-69-f05fc7974a4d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['QUANTITY_IN_100KG'] = df1['QUANTITY_IN_100KG'].astype(int)


False

Dentro de los datos, en la columna PERIOD se encuentran datos que recogen el total de importaciones y exportaciones en un país. Esto se encuentra en el valor 'Jan.-Dec. AÑO', siendo *AÑO* sustituible por el año que estemos leyendo. Estos valores los eliminamos ya que no nos son útiles, pues estamos más interesados en los valores de forma mensual.

In [ ]:
df1.set_index(['PERIOD'], inplace=True)

df1 = df1.drop('Jan.-Dec. 2018')
df1 = df1.drop('Jan.-Dec. 2019')
df1 = df1.drop('Jan.-Dec. 2020')

df1 = df1.reset_index()

Traducimos los productos que se encontraban en inglés para facilitarnos su uso y su integración con otros datasets. Además hemos eliminado la columna Partner porque solo tenía un valor y lo hemos considerado innecesario 

In [ ]:
counts = df1.groupby("PRODUCT").size()
for i in counts.index:
  df1.PRODUCT= df1.PRODUCT.replace({i:translator.translate(i)})
df1 = df1.drop(columns=['PARTNER'])

Comprobamos que los tipos de los datos están correctamente, como vemos que no lo están, decidimos cambiar los tipos de las columnas que están mal.

In [ ]:
df1.dtypes

PERIOD                 object
REPORTER             category
PRODUCT                object
FLOW                 category
VALUE_IN_EUROS          int64
QUANTITY_IN_100KG       int64
dtype: object

In [ ]:
# Normalizamos el nombre de las columnas y sus tipos para poder trabajar correctamente con otros datasets
df1 = df1.convert_dtypes()

df1.rename(columns = {'PERIOD' : 'fecha',
                      'REPORTER' : 'pais',
                      'PRODUCT' : 'producto',
                      'FLOW' : 'flow',
                      'VALUE_IN_EUROS' : 'valor en euros',
                      'QUANTITY_IN_100KG' : 'cantidad en 100Kg'}, inplace = True)

# Hay alguna columna que no se cambia al usar el metodo convert_dtypes por lo que hay que cambiarla a mano
df1['pais'] = df1['pais'].astype('string')

In [ ]:
df1.dtypes

fecha                  string
pais                   string
producto               string
flow                 category
valor en euros          Int64
cantidad en 100Kg       Int64
dtype: object

La columna 'fecha' contiene las fechas como 'Mes. Año' lo cual no nos es útil para comparar valores con el resto de datasets, por lo que lo modificamos para que siga el formato 'XX/YYYY' siendo XX el mes y YYYY el año.

In [ ]:
trad_meses = {
    "Jan": "1",
    "Feb": "2",
    "Mar": "3",
    "Apr": "4",
    "May": "5",
    "Jun": "6",
    "Jul": "7",
    "Aug": "8",
    "Sep": "9",
    "Oct": "10",
    "Nov": "11",
    "Dec": "12",
}

def convert_month_to_number(value):
    # Utiliza una expresión regular para buscar el nombre del mes en el valor dado
    regex = r"^(\w+)"
    month_name = re.search(regex, value).group(1)

    # Utiliza el diccionario trad_meses para cambiar el nombre del mes por un número
    month_number = trad_meses[month_name]

    # Devuelve el valor dado con el mes en formato numérico
    return month_number + "/" + value[-4:]

# Aplica la función "convert_month_to_number" a la columna "fecha" usando la función .apply()
df1["fecha"] = df1["fecha"].apply(convert_month_to_number)

## Descarga de dataset resultante
Se tranforma el dataframe a csv separando las celdas por el símbolo ";", que es el símbolo que lee excel por defecto como separador.

In [ ]:
# Ejecute estas líneas de código si quiere descargar el fichero con el dataset resultante

df1.to_csv('df_import_export.csv', sep = ';', index=False)

files.download('df_import_export.csv')

## Profiling del dataset 4

> Bloque con sangría



Hacemos profiling del *cuarto* dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(df1, title="Imports-Exports", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Descripción: Dataset 5


## Carga de Datos: Dataset 5
Carga del **Dataset5_Coronavirus_cases.txt**, cuyo continido son los **casos de Corona virus**, formato de archivo tipo **.txt**. Tipo de saparadores entre los datos **"|"**.

In [ ]:
data5 = pd.read_csv("Dataset5_Coronavirus_cases.txt", sep="|")
data5.columns

Index(['dateRep', 'day', 'month', 'year', 'cases', 'deaths',
       'countriesAndTerritories', 'geoId', 'countryterritoryCode',
       'popData2019', 'continentExp',
       'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'],
      dtype='object')

Columnas descartadas: 
- **'day'**: innecesaria, ya se utilizara **'fecha'** para describir la fecha conforme a las necesidades del estudio.
- **'dateRep'**: redundante, ya se utilizan **'fecha'** para describir la fecha, en un formato más útil para la selección con respecto al estudio a realizar.
- **'geoId'**: irrelevante, es más descriptivo el uso de **'pais'** para visualizar los datos por país.
- **'countryterritoryCode'**: al igual de 'geoId' es una abreviatura del país al que hace referencia.
- **'popData2019'**: irrelevante, para el estudio los valores de la población durante 2019 son innecesario. No se va a comparar la muestra de 2019 con muestras de los años siguientes.
- **'continentExp'**: irrelevante más halla de para darle forma a los datos. Para el estudio solo se van a tomar en cuenta los países de Europa, por lo cual no es necesario hacer referencia al continente de dichos países.

Con respecto a los otros datos (columnas) descartados, se utilizar columnas que se han obtenido como resultado de operar con estas.

Para obtener los datos de la forma deseada se va a operar con los siguientes datos del dataset 5, también serán renombradas las columnas para facilitar el trabajo con los datos:

In [ ]:
data5 = data5[['countriesAndTerritories', 'month', 'year', 'cases', 'deaths', 'continentExp',
               'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000']]

data5.rename(columns = {'countriesAndTerritories' : 'pais',
                        'month' : 'mes',
                        'year' : 'año',
                        'cases' : 'casos',
                        'deaths' : 'muertes',
                        'continentExp' : 'continente',
                        'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000' : 'incicencia_acumulada'
                        }, inplace = True)

data5['fecha'] = data5['mes'].astype(str) + '/' + data5['año'].astype(str)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


## Limpieza de Datos 

Datos incompletos, mal organizados, errores, etc. Hay que intentar identificarlos y tratarlos antes de comenzar el análisis, o mediante borrado, sustitución, etc.

Eliminación de los datos cuyo valor de 'continente' se **distinto de Europa** al encontrarse fuera del rango de valores del estudio.

In [ ]:
data5.set_index(['continente'], inplace=True)

data5 = data5.drop('Africa')
data5 = data5.drop('America')
data5 = data5.drop('Asia')
data5 = data5.drop('Oceania')
data5 = data5.drop('Other')

data5 = data5[['pais', 'fecha', 'casos', 'muertes', 'incicencia_acumulada']]

Conversion de los valores **incidencia_acumulada** de *'object'* a *'float'*. Asi como la eliminar los valores nulos rellenando los campos vacios con **0.0**, debido a que en algunos casos, cuando la **incidencia_acumulada** es menor que 0 esta no viene representada.

In [ ]:
data5['incicencia_acumulada'] = data5['incicencia_acumulada'].str.replace(',', '.').astype(float)
data5['incicencia_acumulada'] = data5['incicencia_acumulada'].fillna(0.0)

Creamos las columnas **'casos_mes'**, **'muertes_mes'** e **'incicencia_media_mes'** necesarias para el estudio: 

In [ ]:
data5.set_index(['pais', 'fecha'], inplace=True)

data5['casos_mes'] = data5.groupby(['pais', 'fecha'])['casos'].sum()
data5['muertes_mes'] = data5.groupby(['pais', 'fecha'])['muertes'].sum()
data5['incicencia_media_mes'] = data5.groupby(['pais', 'fecha'])['incicencia_acumulada'].mean()

Definimos el dataset con las colunas relevantes para el estudio, así como a la vez descartando las que no sean necesarias:

In [ ]:
data5 = data5[['casos_mes', 'muertes_mes', 'incicencia_media_mes']]
data5 = data5.drop_duplicates()

Comprobamos que no quedan valores nulos en el dataset 5:

In [ ]:
data5.isnull().values.any()

False

Para el trabajó con los datos, reestablecemos el valor de **índice** al asignado **por defecto**.

In [ ]:
data5 = data5.reset_index() 

Para mantener el mismo orden que otros datasets, movemos de posición la columna fecha al principio.
Además modificamos el tipo de las columnas de fecha y país.

In [ ]:
data5['fecha'] = data5['fecha'].astype('string')
data5['pais'] = data5['pais'].astype('string')

In [ ]:
columns_d5 = data5.columns.tolist()
columns_d5.insert(0, columns_d5.pop(columns_d5.index('fecha')))
data5 = data5.reindex(columns = columns_d5)

## Descarga de dataset resultante
Se tranforma el dataframe a csv separando las celdas por el símbolo ";", que es el símbolo que lee excel por defecto como separador.

In [ ]:
# Ejecute estas líneas de código si quiere descargar el fichero con el dataset resultante

data5.to_csv('df_casos_covid.csv', sep = ';', index=False)

files.download('df_casos_covid.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Profiling del dataset 5

Hacemos profiling del quinto dataset para poder analizar mejor los datos contenidos en el dataset final una vez lo hemos limpiado.

In [ ]:
profile = ProfileReport(data5, title="Coronavirus cases", html={'style':{'full_width': True}})
profile.to_notebook_iframe()

# Tarjetas de datos

Aquí se encuentran los procesamientos realizados para la obtención de las distintas tarjetas de datos para las hipótesis. Únicamente se encuentra los procesos para las tarjetas de datos 1 y 3, debido a que la tarjeta de datos 2 se obtiene tras los procesos realizados en la limpieza del dataset 2. De todos modos, se encontrará abajo un pequeño subapartado en el que se puede descargar la tarjeta, pero es necesario haber ejecutado la parte de limpieza del apartado **'Dataset 2 - Consumo comunidades'**

## Creación dataset para H1

Este apartado consiste en unir los datasets 4 y 5 con el fin de crear un nuevo dataset para la hipótesis número 1 del proyecto.

La operación consiste en unir la información de ambos basándonos en los valores para las columnas fecha y país.

In [ ]:
h1 = pd.merge(df1, data5, on=['fecha', 'pais'], how='inner')
h1.head()

,fecha,pais,producto,flow,valor en euros,cantidad en 100Kg,casos_mes,muertes_mes,incicencia_media_mes
0,2/2020,Austria,"Bananos, frescos (exc. ​​plátanos) (2012-2500)",IMPORT,0,0,7,0,0.006228
1,2/2020,Austria,"Bananos, frescos (exc. ​​plátanos) (2012-2500)",EXPORT,0,0,7,0,0.006228
2,2/2020,Austria,"Coles de Bruselas, frescas o refrigeradas (198...",IMPORT,0,0,7,0,0.006228
3,2/2020,Austria,"Coles de Bruselas, frescas o refrigeradas (198...",EXPORT,0,0,7,0,0.006228
4,2/2020,Austria,"Pepinos, frescos o refrigerados (1998-2500)",IMPORT,2148605,12912,7,0,0.006228


Comprobamos que no haya duplicados en caso de que tengamos que corregirlo.

In [ ]:
h1.isnull().values.any()

False

In [ ]:
h1.to_csv('tarjeta_datos_1.csv', sep = ';', index=False)

files.download('tarjeta_datos_1.csv')

## Creación tarjeta de datos 2

En este subapartado se puede descargar la tarjeta de datos 2, obtenida al ejecutar por completo el apartado 'Dataset 2'.

In [ ]:
ds_clean.to_csv('tarjeta_datos_2.csv', sep = ';', index = False)

files.download('tarjeta_datos_2.csv')

## Creación tarjeta de datos 3

Este apartado consiste en unir los datasets 1 y 5 con el fin de crear un nuevo dataset para las hipótesis número 3 y 4 del proyecto.

Lo primero que tenemos que hacer es filtrar los datos del dataset 1 de cajamar para obtener sólo su fecha y del dataset 5 de cajamar para sacar la fecha y los casos por mes de España. Una vez extraidos los mezclamos con un merge y lo almacenamos en df_merged.

Con este dataframe tenemos únicamente las fechas en las que ha habido casos de Covid en España junto con el número total de casos en dicho mes para así poder juntar estos dataframes con el dataset 1, obteniendo el dataset h3. El inconveniente al juntar y obtener el dataset h3 es que podemos tener valores nulos para las fechas en las que no hay casos de Covid, por lo que tenemos que controlar eso además de borrar los duplicados que haya.

En este dataset tenemos la información de las ventas en España junto al número de casos de Covid que ha habido en cada mes.

In [ ]:
df1_fecha = df1_final['fecha']
df5_españa = data5.loc[data5['pais'] == 'Spain']
df2_casos_mes = df5_españa[['fecha', 'casos_mes']]

df_merged = pd.merge(df1_fecha, df2_casos_mes, left_on='fecha', right_on='fecha')

h3 = pd.merge(df1_final, df_merged, on='fecha', how='left')

h3 = h3.drop_duplicates()
h3 = h3.fillna(0)

h3.head()

,fecha,producto,precio_medio_kg,consumo_per_capita,casos_mes
0,1/2018,PATATAS,0.78,2.38,0.0
1,1/2018,TOMATES,1.69,0.88,0.0
2,1/2018,CEBOLLAS,1.01,0.57,0.0
3,1/2018,AJOS,4.17,0.06,0.0
4,1/2018,COLES,1.08,0.15,0.0


In [ ]:
# Comprobamos que no hay nulos
h3.isnull().values.any()

False

Procedemos a descargar la tarjeta de datos para las hipótesis 3 y 4 generadas en este apartado.

In [ ]:
h3.to_csv('tarjeta_datos_3_4.csv', sep = ';', index=False)

files.download('tarjeta_datos_3_4.csv')